In [1]:
import numpy as np
import pandas as pd

In [5]:
tutorial = pd.read_csv("tutorial_filtered.csv")

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
tutorial.shape

(993059, 50)

# 2. Split train/test
## 1. tutorial 로 ad_id별 day_0 추출

In [9]:
grp = tutorial.groupby('ad_id')
first_access = pd.DataFrame(grp['event_datetime'].min())

,event_datetime
ad_id,
00018501-6eee-45e9-b5b5-7b50f966abd8,2021-04-15 03:56:13.665
0001edb6-e7e7-4f74-8993-2bfc9347a959,2021-04-23 19:45:27.894
0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,2021-04-19 17:51:50.317
000289c6-779a-4b82-bab0-652fa477be1f,2021-04-09 15:58:13.805
0002aec5-2842-4fa8-9b13-004ec60a5a2a,2021-04-02 17:46:52.852
...,...
fffe4cf6-ea7b-4377-af60-a019e460bd30,2021-04-22 22:16:08.435
fffe5e96-fb11-462b-bb81-f3bb1911f618,2021-04-11 18:23:59.781
ffffaa74-daa9-4fe1-a4bb-9e9c5853246f,2021-04-23 15:54:31.548


In [10]:
first_access.reset_index(inplace=True)
first_access

In [13]:
first_access.rename(columns={'event_datetime':'day0'}, inplace=True)

## 2. ads 의 date와 day_0로 day계산

In [54]:
ads1 = pd.read_csv("data/brazil_ads_1to5.csv")
ads2 = pd.read_csv("data/brazil_ads_6to10.csv")

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,3,40,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
ads1.columns

Index(['items', 'traffic_source_medium', 'traffic_source_source',
       'traffic_source_name', 'event_previous_timestamp', 'ga_session_number',
       'ga_session_number_timestamp', 'user_level', 'user_level_timestamp',
       'frist_version', 'frist_version_timestamp', 'cuid', 'cuid_timestamp',
       'firebase_exp_8', 'firebase_exp_8_timestamp', 'firebase_exp_10',
       'firebase_exp_10_timestamp', 'ad_id', 'ad_id_timestamp',
       'firebase_exp_12', 'firebase_exp_12_timestamp', 'ga_session_id',
       'ga_session_id_timestamp', 'firebase_exp_4', 'firebase_exp_4_timestamp',
       'firebase_exp_6', 'firebase_exp_6_timestamp', 'first_open_time',
       'first_open_time_timestamp', 'firebase_exp_13',
       'firebase_exp_13_timestamp', 'firebase_exp_14',
       'firebase_exp_14_timestamp', 'firebase_exp_2',
       'firebase_exp_2_timestamp', 'firebase_exp_3',
       'firebase_exp_3_timestamp', 'firebase_last_notification',
       'firebase_last_notification_timestamp', 'last_gclid',

In [56]:
sub_ads1 = ads1[['ad_id', 'event_timestamp', 'gubun', 'state']]
sub_ads2 = ads2[['ad_id', 'event_timestamp', 'gubun', 'state']]

In [57]:
sub_ads1_datetime = pd.to_datetime(sub_ads1['event_timestamp'], unit='us')
sub_ads1.loc[:, 'event_datetime'] = sub_ads1_datetime

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [58]:
sub_ads2_datetime = pd.to_datetime(sub_ads2['event_timestamp'], unit='us')
sub_ads2.loc[:, 'event_datetime'] = sub_ads2_datetime

In [59]:
sub_ads1.shape

(9501099, 5)

In [60]:
sub_ads2.shape

(9498999, 5)

In [61]:
ads = pd.concat([sub_ads1, sub_ads2], ignore_index=True)
ads.shape

In [62]:
len(ads['ad_id'].unique())

243416

In [63]:
ads['ad_id'].isnull().sum()

165586

In [64]:
first_access.shape

(240911, 2)

In [178]:
df = pd.merge(ads, first_access, how='inner', on='ad_id')
df.shape

Index(['ad_id', 'event_timestamp', 'gubun', 'state', 'event_datetime', 'day0'], dtype='object')

In [179]:
df['day0'] = pd.to_datetime(df['day0'])
df.info()

In [181]:
df.loc[:, 'day_x'] = ((df['event_datetime'].dt.date) - (df['day0'].dt.date)).dt.days
df.head()

In [183]:
df[df['day_x']<0].head()

,ad_id,event_timestamp,gubun,state,event_datetime,day0,day_x
258323,5855c472-ec5d-4e3a-ad67-046d17817dfc,1617403180278000,rv,view,2021-04-02 22:39:40.278,2021-04-03 00:56:04.440,-1
258335,5855c472-ec5d-4e3a-ad67-046d17817dfc,1617403506198000,rv,view,2021-04-02 22:45:06.198,2021-04-03 00:56:04.440,-1
258336,5855c472-ec5d-4e3a-ad67-046d17817dfc,1617403574549000,ba,imps,2021-04-02 22:46:14.549,2021-04-03 00:56:04.440,-1
258349,5855c472-ec5d-4e3a-ad67-046d17817dfc,1617403249303000,ba,imps,2021-04-02 22:40:49.303,2021-04-03 00:56:04.440,-1
258357,5855c472-ec5d-4e3a-ad67-046d17817dfc,1617403422475000,ba,imps,2021-04-02 22:43:42.475,2021-04-03 00:56:04.440,-1


In [71]:
not_new_user = df[df['day_x']<0]['ad_id'].unique()

In [72]:
df1 = df[~df['ad_id'].isin(not_new_user)]
df1.shape

(18718875, 7)

In [73]:
len(df1['ad_id'].unique())

240736

In [74]:
df1['day_x'].min()

0

In [75]:
df1['day_x'].max()

44

In [208]:
df1 = df1[df1['state']=='imps']

In [209]:
df2 = pd.DataFrame(df1.groupby(['ad_id', 'day_x'])['state'].count())
df2.head()

state
ad_id                                day_x       
00018501-6eee-45e9-b5b5-7b50f966abd8 0          1
0001edb6-e7e7-4f74-8993-2bfc9347a959 0         19
0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7 0          6
                                     1         11
000289c6-779a-4b82-bab0-652fa477be1f 0          6

In [210]:
df3 = df2.unstack().reset_index()
df3.head()

ad_id state                            \
day_x                                           0     1   2   3   4   5   6   
0      00018501-6eee-45e9-b5b5-7b50f966abd8   1.0   NaN NaN NaN NaN NaN NaN   
1      0001edb6-e7e7-4f74-8993-2bfc9347a959  19.0   NaN NaN NaN NaN NaN NaN   
2      0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7   6.0  11.0 NaN NaN NaN NaN NaN   
3      000289c6-779a-4b82-bab0-652fa477be1f   6.0   NaN NaN NaN NaN NaN NaN   
4      0002aec5-2842-4fa8-9b13-004ec60a5a2a   6.0   NaN NaN NaN NaN NaN NaN   

               ...                                          
day_x   7   8  ...  34  35  36  37  38  39  40  41  42  43  
0     NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1     NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
2     NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
3     NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
4     NaN NaN  ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 45 columns]

In [211]:
df3.shape #(240736, 46)

(225097, 45)

In [212]:
ads_df = df3['state']

In [213]:
ads_df.loc[:, 'ad_id'] = df3['ad_id']
ads_df.head()

day_x,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,ad_id
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00018501-6eee-45e9-b5b5-7b50f966abd8
1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0001edb6-e7e7-4f74-8993-2bfc9347a959
2,6.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7
3,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000289c6-779a-4b82-bab0-652fa477be1f
4,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0002aec5-2842-4fa8-9b13-004ec60a5a2a


In [214]:
ads_df[0].isnull().sum() 
# total -> 351   # imps -> 3792

3792

In [215]:
ads_df['cum_imps'] = ads_df.loc[:, [2,3,4,5,6,7]].sum(axis=1)

In [216]:
ads_df.head()

day_x,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,ad_id,cum_imps
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00018501-6eee-45e9-b5b5-7b50f966abd8,0.0
1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0001edb6-e7e7-4f74-8993-2bfc9347a959,0.0
2,6.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,0.0
3,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,000289c6-779a-4b82-bab0-652fa477be1f,0.0
4,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0002aec5-2842-4fa8-9b13-004ec60a5a2a,0.0


## 3. day 2-7 누적 imps 값이 1 이상인 사람 = 1 / 0인 사람 = 0

→ 이 기준으로 비율맞춰 train, test split 된 total ad_id, train(1)/test(0) dataframe 생성하고 공유

In [218]:
ads_df['cum_imps_YN'] = 0
ads_df.loc[ads_df['cum_imps'] != 0, 'cum_imps_YN'] = 1

In [219]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1004)

ads_df['train'] = 0
for train_idx, test_idx in split.split(ads_df, ads_df["cum_imps_YN"]):
    ads_df.loc[train_idx, 'train'] = 1

In [220]:
len(df_strat_train[df_strat_train['cum_imps_YN']==1])/len(df_strat_train)

0.2513552246245872

In [221]:
len(df_strat_test[df_strat_test['cum_imps_YN']==1])/len(df_strat_test)

0.25135000415385894

In [222]:
ads_df.head()

day_x,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,ad_id,cum_imps,cum_imps_YN,train
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,00018501-6eee-45e9-b5b5-7b50f966abd8,0.0,0,1
1,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0001edb6-e7e7-4f74-8993-2bfc9347a959,0.0,0,0
2,6.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,0.0,0,1
3,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,000289c6-779a-4b82-bab0-652fa477be1f,0.0,0,1
4,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0002aec5-2842-4fa8-9b13-004ec60a5a2a,0.0,0,1


In [223]:
ads_df.reset_index(drop=True, inplace=True)

In [224]:
ads_df = ads_df[['ad_id', 'cum_imps', 'cum_imps_YN', 'train']]

In [225]:
ads_df.head()

day_x,ad_id,cum_imps,cum_imps_YN,train
0,00018501-6eee-45e9-b5b5-7b50f966abd8,0.0,0,1
1,0001edb6-e7e7-4f74-8993-2bfc9347a959,0.0,0,0
2,0001ef07-c27e-4d91-a8bb-d0aa5f0d45e7,0.0,0,1
3,000289c6-779a-4b82-bab0-652fa477be1f,0.0,0,1
4,0002aec5-2842-4fa8-9b13-004ec60a5a2a,0.0,0,1


In [226]:
ads_df.to_csv('real_train_YN.csv', index = False)

### 노출 성공 여부 고려 X / 노출 성공 여부 고려 O 비교

In [228]:
real_df = pd.read_csv("real_train_YN.csv")
df = pd.read_csv("train_YN.csv")

In [230]:
real_df[real_df['train']==1].shape

(180077, 4)

In [231]:
df[df['train']==1].shape

(192588, 4)